# Información tabular: introducción a `pandas`

Si has buscado información sobre el uso de `Python` para el manejo de datos, seguramente te has topado con el nombre `pandas`. `pandas` es el nombre de uno de los módulos más indispensables, que da métodos y herramientas para el procesamiento de información tabular. Fue creada en 2008 por Wes McKinney, por lo que a estas alturas ya es un módulo sumamente maduro. Como en muchos otros problemas de programación, **Stack Overflow** es el primer sitio donde puedes buscar ayuda sobre tus problemas. Algunos libros recomendados son **Python for Data Analysis** y **Learning the Pandas Library** (ambos en el canal de lecturas recomendadas), aunque por supuesto no hay mejor sitio para consultar tus dudas sobre cómo usar una función o método que la propia documentación de la librería.

Sin más preámbulo, exploremos las estructuras más básicas y vayamos escalando en complejidad.

In [1]:
import pandas as pd

## La base: `pd.Series`

Recordarás que en la primera sesión hablamos del tipo `pd.Series`. Es la base en `pandas`, y es la cruza entre una lista y un diccionario. Los ítems que las conforman están almacenadas en orden, y hay etiquetas para recuperarlos. Los elementos de una serie son:

<img src="https://github.com/ArturoBell/py_biol/blob/assets/slides/c04_series.png?raw=true" style="width:1000px">

Para crearlas solo pasaremos una lista de valores. Al hacerlo así, `pandas` automáticamente asigna un índice (`index`), y establece el nombre de la serie como `None`.

Como toda función, siempre es recomendable ver la ayuda antes de comenzar a utilizarla:

In [2]:
pd.Series?

Init signature:
pd.Series(
    data=None,
    index=None,
    dtype=None,
    name=None,
    copy=False,
    fastpath=False,
)
Docstring:     
One-dimensional ndarray with axis labels (including time series).

Labels need not be unique but must be a hashable type. The object
supports both integer- and label-based indexing and provides a host of
methods for performing operations involving the index. Statistical
methods from ndarray have been overridden to automatically exclude
missing data (currently represented as NaN).

Operations between Series (+, -, /, *, **) align values based on their
associated index values-- they need not be the same length. The result
index will be the sorted union of the two indexes.

Parameters
----------
data : array-like, Iterable, dict, or scalar value
    Contains data stored in Series.

    .. versionchanged:: 0.23.0
       If data is a dict, argument order is maintained for Python 3.6
       and later.

index : array-like or Index (1d)
    Values must 

### Creación

Creemos entonces una serie utilizando la lista de especies de la diapositiva:

In [3]:
especies = ['Stenella attenuata', 'Stenella longirostris',
            'Stenella coeruleoalba', 'Stenella frontalis']
pd.Series(especies)

0       Stenella attenuata
1    Stenella longirostris
2    Stenella coeruleoalba
3       Stenella frontalis
dtype: object

Podemos pasar también un diccionario, en cuyo caso las claves pasan a formar el índice:

In [4]:
especies = {'Stenella': 'attenuata',
            'Orcinus': 'orca',
            'Lagenorhynchus': 'obliquidens',
            'Pseudorca': 'crassidens'
            }
s = pd.Series(especies)
s

Stenella            attenuata
Orcinus                  orca
Lagenorhynchus    obliquidens
Pseudorca          crassidens
dtype: object

Podemos obtener los índices con el **atributo** `index`:

In [5]:
s.index

Index(['Stenella', 'Orcinus', 'Lagenorhynchus', 'Pseudorca'], dtype='object')

Y los valores con `values`:

In [6]:
s.values

array(['attenuata', 'orca', 'obliquidens', 'crassidens'], dtype=object)

Otra forma es pasar el índice de manera explícita como una lista (o tuple):

In [7]:
s = pd.Series(['attenuata', 'orca', 'obliquidens', 'crassidens'],
              index = ['Stenella', 'Orcinus', 'Lagenorhynchus', 'Pseudorca'])
s

Stenella            attenuata
Orcinus                  orca
Lagenorhynchus    obliquidens
Pseudorca          crassidens
dtype: object

### Indización/consulta

Podemos acceder a los elementos de una serie ya sea por posición o etiqueta del índice. ¿Qué pasa si no le das un índice a la serie? En ese caso tanto la posición como la etiqueta son iguales; sin embargo, es importante tener en cuenta que:

- Para indizar por ubicación numérica, empezaremos en 0 y utilizaremos el atributo `.iloc`
- Para indizar por la etiqueta del índice, utilizaremos el atributo `.loc`.

Es importante notar que tanto `.iloc` como `.loc` son atributos, por lo que se indizan con `[]`:

In [8]:
s.iloc[3]

'crassidens'

In [9]:
s.loc['Pseudorca']

'crassidens'

Podemos indizar de manera directa:

In [10]:
s[3]

'crassidens'

In [11]:
s['Pseudorca']

'crassidens'

¿Por qué utilizar entonces los atributos? Para evitar errores. Tomemos la siguiente serie:

In [12]:
abund = pd.Series({100: 'Stenella',
                   101: 'Orcinus',
                   102: 'Lagenorhynchus',
                   103: 'Pseudorca'})

Si intentamos indizarla como `abund[0]` vamos a tener un error (`KeyError: 0`), ya que no llamará a `abund.iloc[0]` como nosotros esperamos, sino que buscará la entrada cuyo índice sea 0, que no existe en la colección.

In [13]:
#abund[0]

¿Mi recomendación? Procura utilizar `.iloc` y `.loc` para evitar confusiones y sorpresas. Otra cosa a tener en cuenta es que el atributo `.loc` permite no solo indizar, sino también añadir nueva información. Si el valor pasado como índice no existe, genera una nueva entrada. También es relevante mencionar que los índices pueden a) tener valores repetidos y b) tener tipos mixtos. En este último caso, `pandas` automáticamente cambia los tipos de `NumPy` según sea necesario. Volviendo a nuestra Serie `abund`, añadamos una nueva entrada donde el índice sea `Stenella` y el valor `attenuata`:

In [14]:
s.loc['Stenella'] = 'attenuata'
s

Stenella            attenuata
Orcinus                  orca
Lagenorhynchus    obliquidens
Pseudorca          crassidens
dtype: object

### Operaciones

Al ser colecciones de elementos, podemos pensar en iterar sobre la colección; sin embargo, esta aproximación es lenta. En su lugar tenemos aproximaciones **vectorizadas**, que son mucho más eficientes. Hagamos una comparación en la que el objetivo sea sumar cuatro números repetidos 10000 veces:

In [15]:
import numpy as np
s = pd.Series(np.repeat([100, 200, 120, 3., 101.5], 10000))
s

0        100.0
1        100.0
2        100.0
3        100.0
4        100.0
         ...  
49995    101.5
49996    101.5
49997    101.5
49998    101.5
49999    101.5
Length: 50000, dtype: float64

Primero la aproximación iterativa. Para medir el tiempo utilizaremos la función "mágica" (así se les llama) `%%timeit -n 1000`; es decir, ejecutará el código 1000 veces y la salida será el tiempo total promedio por corrida y su desviación estándar:

In [16]:
%%timeit -n 1000
total = 0
for item in s:
    total+=item

4.72 ms ± 84.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


<div class = "alert alert-block alert-info">
    <p>El operador <code>+=</code> le suma <code>item</code> a <code>total</code>, y actualiza el valor de total. Esta estructura se puede utilizar con el resto de operadores aritméticos (-, *, /, **)</p></div>

Luego la aproximación vectorizada:

In [17]:
%%timeit -n 1000
np.sum(s)

110 µs ± 5.47 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Notarás que es una diferencia muy importante. 112 **micro** segundos en la vectorizada vs. 4.63 **mili** segundos en la aproximación iterativa. Esta diferencia, además, se va a hacer más marcada entre más compleja sea la operación y más información tengamos. ¿Esto quiere decir que las iteraciones no vectorizadas no tienen lugar? En absoluto. Simplemente significa que, si podemos hacer nuestro código eficiente (y nos interesa el tiempo de ejecución), vale la pena buscar una alternativa vectorizada.

Además de esto, `pandas` permite "transmitir" (`broadcast`) una operación; es decir, aplicarla a cada valor en la serie mientras lo cambia:

In [18]:
s+=2 # Añade 2 a cada item y lo cambia
s.head()

0    102.0
1    102.0
2    102.0
3    102.0
4    102.0
dtype: float64

Como podrás imaginarte, esta estructura es el equivalente a una columna o renglón de una de una tabla y, de hecho, cada columna/renglón de un `DataFrame` es una serie.

## El objeto principal: `pd.DataFrame`

Si la serie es el objeto base, el principal es el `DataFrame`. Antes mencionamos que esta es una estructura bi-dimensional de Series, donde hay un índice y múltiples columnas de contenido (cada una con una etiqueta):

<img src="https://github.com/ArturoBell/py_biol/blob/assets/slides/c04_DataFrame.png?raw=true" style="width:1000px">

### Creación

Como puedes ver en la diapositiva, un `DataFrame` puede ser accesado de distintas maneras, pero primero veamos las formas de crearlo:

- Un grupo de Series, donde cada serie representa un renglón de los datos
- Un grupo de diccionarios, donde cada uno también representa un renglón de datos

Veamos un ejemplo:

In [19]:
auto1 = pd.Series({'Marca': 'Mazda',
                   'Modelo': 3,
                   'Motor': '2.5L'})
auto2 = pd.Series({'Marca': 'Mazda',
                   'Modelo': 2,
                   'Motor': '2.0L'})
auto3 = pd.Series({'Marca': 'Mazda',
                   'Modelo': 'CX-3',
                   'Motor': '2.0L'})
autos = pd.DataFrame([auto1, auto2, auto3],
                     index = ['Auto1', 'Auto2', 'Auto3'])
autos

,Marca,Modelo,Motor
Auto1,Mazda,3,2.5L
Auto2,Mazda,2,2.0L
Auto3,Mazda,CX-3,2.0L


Una forma más eficiente sería utilizando un diccionario, donde las claves correspondan a los nombres de las columnas, y los valores sean listas con los datos de cada columna:

In [20]:
autos = pd.DataFrame({'Marca': ['Mazda', 'Mazda', 'Mazda'],
                      'Modelo': [3, 2, 'CX3'],
                      'Motor': ['2.5L', '2.0L', '2.0L']},
                     index = ['Auto1', 'Auto2', 'Auto3'])
autos

,Marca,Modelo,Motor
Auto1,Mazda,3,2.5L
Auto2,Mazda,2,2.0L
Auto3,Mazda,CX3,2.0L


Similar a las Series, podemos extraer la información utilizando los atributos `.loc` y `.iloc`; sin embargo, ya que los DataFrames son bi-dimensionales, pasar un solo valor a `.loc` va a regresar una serie si solo hay un renglón a regresar

In [21]:
autos.loc['Auto2']

Marca     Mazda
Modelo        2
Motor      2.0L
Name: Auto2, dtype: object

Como habíamos mencionado, cada dimensión del DataFrame es una serie:

In [22]:
type(autos['Marca'])

pandas.core.series.Series

In [23]:
type(autos.loc['Auto2'])

pandas.core.series.Series

### Indización

Con esto también habrás notado que el operador de indización (`[]`) está reservado para el nombre de las columnas. ¿Y si quiero acceder por el número de columna? Entonces utilizamos el operador `.iloc`:

In [24]:
autos.iloc[:,1]

Auto1      3
Auto2      2
Auto3    CX3
Name: Modelo, dtype: object

Ya que es un objeto bi-dimensional, podemos especificar qué elementos queremos extraer. En el ejemplo anterior quisimos extraer todos los renglones (operador `:`) de la segunda columna (recuerda, el valor del primer índice es 0) y por eso indicamos `[:, 1]`. Si queremos extraer solo un intervalo podemos hacerlo también (operador `i:f`):

In [25]:
autos.iloc[2, 0:2]

Marca     Mazda
Modelo      CX3
Name: Auto3, dtype: object

En este ejemplo extrajimos el renglón número 3 (índice 2) y nos quedamos solo con los primeros tres elementos. ¿Cómo realizamos operaciones? Exactamente igual que con las Series. A final de cuentas, cada fragmento (`slice`) que obtengamos del DataFrame es una serie.

### Eliminar datos

Un poco más interesante y relevante es el cómo eliminar (drop) datos de series o DataFrames, aunque es sumamente sencillo: utilizando la función `drop`. Esta función, a diferencia de los métodos `append`, `insert` y `extend` para listas, NO cambia el objeto, sino que regresa una copia sin los renglones (o columnas) que se eliminaron:

In [26]:
autos.drop('Auto1')

,Marca,Modelo,Motor
Auto2,Mazda,2,2.0L
Auto3,Mazda,CX3,2.0L


In [27]:
autos

,Marca,Modelo,Motor
Auto1,Mazda,3,2.5L
Auto2,Mazda,2,2.0L
Auto3,Mazda,CX3,2.0L


Por esta razón, se recomienda primero crear una copia y luego realizar la eliminación:

In [28]:
autos_copia = autos.copy()
autos_copia = autos_copia.drop('Auto1')
autos_copia

,Marca,Modelo,Motor
Auto2,Mazda,2,2.0L
Auto3,Mazda,CX3,2.0L


Aunque no se recomienda mas que para casos excepcionales, podemos utilizar el argumento `inplace = T` para no hacer la copia:

In [29]:
autos.drop('Auto2', inplace = True)
autos

,Marca,Modelo,Motor
Auto1,Mazda,3,2.5L
Auto3,Mazda,CX3,2.0L


Otro parámetro interesante de la función es `axis`, en el cuál indicaremos en qué eje queremos realizar la eliminación. Por defecto trabaja sobre los renglones (`axis = 0` o `axis = 'index'`, aunque podemos trabajar también sobre las columnas (`axis = 1` o `axis = 'columns'`):

In [30]:
autos.drop('Modelo', axis = 'columns')

,Marca,Motor
Auto1,Mazda,2.5L
Auto3,Mazda,2.0L


### Cargar datos tabulares

Otro proceso sumamente cotidiano es el leer nuestros datos, almacenados en archivos de texto separados por comas o tabulaciones, u hojas de cálculo de Excel, por ejemplo. `pandas` cuenta con herramientas justo para eso. Carguemos los datos `fq.txt`, que contienen mediciones de parámetros fisicoquímicos en distintos sitios de muestreo. Este es un archivo separado por tabulaciones, por lo que utilizaremos la función `pd.read_table('data/fq.txt')`. Además, la primera columna contiene los identificadores de los sitios, por lo que debe asignarse al índice del `DataFrame`, lo cual haremos con el argumento `index_col = 0`:

In [31]:
fq = pd.read_table('data/fq.txt', index_col = 0)

Ahora veamos los primeros 6 elementos (el encabezado), con el método `.head()`:

In [32]:
fq.head()

,Temp,NH4,NO3,OD,Prof,Trans,Caud,SST,STD,PO4,DBO5,DQO
S1,16.9,0.78,1.83,7.50,27,30,1.01,10.0,86.9,0.6,23.27,29
S2,17.2,0.75,2.44,6.05,56,56,0.68,4.0,87.9,0.4,24.11,5
S3,17.4,1.25,1.60,5.84,37,29,1.53,6.5,101.5,0.6,20.63,15
S4,18.0,0.54,2.95,7.65,122,50,2.46,4.5,91.7,0.7,25.19,7
S5,19.2,0.88,2.70,4.07,137,18,6.58,50.0,95.7,0.2,24.83,16


Si quisiéramos cargar el archivo `da.csv`, que es un archivo separado por comas, lo haríamos con la función `pd.read_csv('data/da.csv'`). Este archivo contiene datos de longitud total a distintas edades de delfines. Aquí no hay un índice, por lo que no lo indicaremos al cargar los datos:

In [33]:
da = pd.read_csv('data/da.csv')
da.head()

,age,lt
0,0,111.168190
1,0,106.965427
2,0,110.136387
3,0,109.639519
4,0,112.362961


Con un archivo Excel es la misma historia (intenta adivinar la función), así que no lo demostraremos para no hacer el cuento largo. En su lugar, realicemos algunas operaciones comunes.

### Máscaras booleanas

Una máscara booleana es una forma elegante de llamarle a un filtro, que dará `True` si se cumple la condición especificada y `False` en caso contrario. Este filtro es un arreglo de datos (de 1 o 2 dimensiones) donde hay un valor booleano para cada valor de la columna o del renglón de interés. La idea es que ese filtro, esencialmente, se sobrepone a la estructura de datos de interés. Si el valor es `True`, el valor será extraído, si es `False`, no. Imaginemos que en `fq` nos interesa saber qué sitios tuvieron una temperatura menor a 20ºC. Para esto podemos hacer algo tan simple como:

In [34]:
fq['Temp'] < 20

S1      True
S2      True
S3      True
S4      True
S5      True
S6      True
S7      True
S8      True
S9      True
S10     True
S11     True
S12    False
S13    False
S14    False
S15    False
S16    False
S17    False
S18    False
S19    False
S20    False
S21    False
Name: Temp, dtype: bool

Pero podemos también extraer solo los sitios donde el resultado es `True`. Para esto primero podemos guardar la máscara (`mask`, una serie, en este caso) en un objeto y luego utilizar el método `.where(mask)` para filtrar los datos; sin embargo, esta operación asignará NaN (Not a Number) en todos los casos donde la máscara tenga `False`, por lo que podemos encadenar el método `dropna()` para eliminarlos:

In [35]:
mask = fq['Temp'] < 20
cold = fq.where(mask).dropna()
cold

,Temp,NH4,NO3,OD,Prof,Trans,Caud,SST,STD,PO4,DBO5,DQO
S1,16.9,0.78,1.83,7.50,27.0,30.0,1.01,10.000000,86.9,0.60,23.27,29.0
S2,17.2,0.75,2.44,6.05,56.0,56.0,0.68,4.000000,87.9,0.40,24.11,5.0
S3,17.4,1.25,1.60,5.84,37.0,29.0,1.53,6.500000,101.5,0.60,20.63,15.0
S4,18.0,0.54,2.95,7.65,122.0,50.0,2.46,4.500000,91.7,0.70,25.19,7.0
S5,19.2,0.88,2.70,4.07,137.0,18.0,6.58,50.000000,95.7,0.20,24.83,16.0
S6,19.3,0.98,2.50,3.74,168.0,18.0,8.42,21.000000,102.2,0.57,23.09,7.0
S7,19.6,0.90,1.53,3.61,148.0,15.0,6.20,112.000000,104.1,0.87,21.17,18.0
S8,17.5,0.97,3.96,8.01,45.0,6.0,3.31,1163.333333,104.1,0.00,23.27,52.0
S9,18.2,0.95,3.12,6.57,134.0,5.0,10.52,298.666667,104.4,0.00,59.82,62.0
S10,18.6,0.98,2.55,4.61,174.0,5.0,7.66,572.000000,104.3,0.00,35.53,23.0


<div class = "alert alert-block alert-info">
    <p>Recuerda: encadenar consiste en pasar la información que está a la izquierda hacia la derecha directamente (sin objeto intermediario). Es posible SOLO cuando los métodos y atributos son compatibles con dicho encadenamiento. Cuando encadenamos métodos y atributos el operador que utilizaremos es <code>.</code>, mientras que si encadenamos operaciones lógicas utilizaremos algún operador lógico.</p></div>

Con estos datos filtrados podemos tal vez obtener el número de elementos en cada columna con el método `.count()`:

In [36]:
cold.count()

Temp     11
NH4      11
NO3      11
OD       11
Prof     11
Trans    11
Caud     11
SST      11
STD      11
PO4      11
DBO5     11
DQO      11
dtype: int64

Que es diferente al número de elementos en la base completa:

In [37]:
fq.count()

Temp     21
NH4      21
NO3      21
OD       21
Prof     21
Trans    21
Caud     21
SST      21
STD      21
PO4      21
DBO5     21
DQO      21
dtype: int64

Otra cosa que es importante conocer es que el indizado en `pandas` admite máscaras booleanas. Al filtrar de esta manera no necesitamos eliminar los `NaN`s:

In [38]:
fq[fq['Temp'] > 20]

,Temp,NH4,NO3,OD,Prof,Trans,Caud,SST,STD,PO4,DBO5,DQO
S12,24.2,0.70,2.18,3.72,155,52,0.52,599.000000,95.4,0.55,21.23,18
S13,20.4,1.03,3.34,3.37,199,4,5.58,94.000000,109.2,0.00,21.29,32
S14,20.8,1.08,3.53,3.80,96,6,1.96,607.594937,107.6,0.25,34.45,43
S15,20.6,0.90,2.05,2.65,76,10,4.64,134.000000,113.7,0.60,22.73,35
S16,21.4,1.56,1.10,2.24,84,13,6.93,88.750000,129.1,0.57,20.21,14
S17,24.4,1.36,0.40,1.34,42,17,0.84,88.000000,137.7,0.45,20.45,36
S18,21.2,0.98,2.58,3.97,131,8,4.19,234.615385,108.0,0.30,22.79,50
S19,22.5,1.57,0.44,1.03,125,7,13.28,160.000000,129.5,0.80,30.34,53
S20,23.8,1.40,2.13,0.91,78,8,15.44,153.333333,136.6,0.25,32.62,61
S21,25.8,1.34,3.13,0.81,176,6,13.57,290.000000,129.2,0.25,34.30,58


¿Y si queremos filtrar a partir de dos condiciones? Podemos empatar dos máscaras utilizando un operador lógico, y el resultado es una nueva máscara. En consecuencia, los operadores y/o (`&`/`|`) los podemos encadenar para crear filtros más complejos, y el resultado es una sola máscara. OJO: al hacer esto nuestras máscaras a encadenar deben de estar entre paréntesis. En este primer ejemplo nos interesan los sitios en los que la temperatura haya sido superior a 20ºC **Y** en los que el caudal haya sido superior a $1\frac{m^3}{s}$; es decir, que cumplan ambas características al mismo tiempo:

In [39]:
mask = (fq['Temp'] > 20) & (fq['Caud'] > 1)
fq[mask]

,Temp,NH4,NO3,OD,Prof,Trans,Caud,SST,STD,PO4,DBO5,DQO
S13,20.4,1.03,3.34,3.37,199,4,5.58,94.000000,109.2,0.00,21.29,32
S14,20.8,1.08,3.53,3.80,96,6,1.96,607.594937,107.6,0.25,34.45,43
S15,20.6,0.90,2.05,2.65,76,10,4.64,134.000000,113.7,0.60,22.73,35
S16,21.4,1.56,1.10,2.24,84,13,6.93,88.750000,129.1,0.57,20.21,14
S18,21.2,0.98,2.58,3.97,131,8,4.19,234.615385,108.0,0.30,22.79,50
S19,22.5,1.57,0.44,1.03,125,7,13.28,160.000000,129.5,0.80,30.34,53
S20,23.8,1.40,2.13,0.91,78,8,15.44,153.333333,136.6,0.25,32.62,61
S21,25.8,1.34,3.13,0.81,176,6,13.57,290.000000,129.2,0.25,34.30,58


En este segundo ejemplo nos interesan los sitios en los que la temperatura haya sido superior a 20ºC **O** en los que el caudal haya sido superior a $1\frac{m^3}{s}$; es decir, que se cumpla al menos una de las características:

In [40]:
mask = (fq['Temp'] > 20) | (fq['Caud'] > 1)
fq[mask]

,Temp,NH4,NO3,OD,Prof,Trans,Caud,SST,STD,PO4,DBO5,DQO
S1,16.9,0.78,1.83,7.50,27,30,1.01,10.000000,86.9,0.60,23.27,29
S3,17.4,1.25,1.60,5.84,37,29,1.53,6.500000,101.5,0.60,20.63,15
S4,18.0,0.54,2.95,7.65,122,50,2.46,4.500000,91.7,0.70,25.19,7
S5,19.2,0.88,2.70,4.07,137,18,6.58,50.000000,95.7,0.20,24.83,16
S6,19.3,0.98,2.50,3.74,168,18,8.42,21.000000,102.2,0.57,23.09,7
S7,19.6,0.90,1.53,3.61,148,15,6.20,112.000000,104.1,0.87,21.17,18
S8,17.5,0.97,3.96,8.01,45,6,3.31,1163.333333,104.1,0.00,23.27,52
S9,18.2,0.95,3.12,6.57,134,5,10.52,298.666667,104.4,0.00,59.82,62
S10,18.6,0.98,2.55,4.61,174,5,7.66,572.000000,104.3,0.00,35.53,23
S11,19.6,1.09,4.21,4.67,79,7,8.10,649.000000,104.3,0.25,35.08,49


### Operaciones con columnas

Otra tarea sumamente común es realizar operaciones sobre los valores de una columna y guardar el resultado en una columna diferente, o puede que querramos solo añadir nuevos valores o, por el contrario, eliminar una columna. Veamos entonces algunas formas de hacerlo. El procedimiento más sencillo es añadir una columna nueva. Simplemente indizaremos el `DataFrame` con el nombre de la nueva columna y le asignaremos sus valores, tal que:

In [41]:
fq['nueva_col'] = None
fq.head()

,Temp,NH4,NO3,OD,Prof,Trans,Caud,SST,STD,PO4,DBO5,DQO,nueva_col
S1,16.9,0.78,1.83,7.50,27,30,1.01,10.0,86.9,0.6,23.27,29,None
S2,17.2,0.75,2.44,6.05,56,56,0.68,4.0,87.9,0.4,24.11,5,None
S3,17.4,1.25,1.60,5.84,37,29,1.53,6.5,101.5,0.6,20.63,15,None
S4,18.0,0.54,2.95,7.65,122,50,2.46,4.5,91.7,0.7,25.19,7,None
S5,19.2,0.88,2.70,4.07,137,18,6.58,50.0,95.7,0.2,24.83,16,None


En este caso, creamos una nueva columna, llamada `nueva_col` y le asignamos el valor `None` y el resultado fue que TODOS los valores de esa columna fueron `None`. Esto pasará siempre que asignemos un solo valor, pero ¿y si queremos asignar toda una columna de valores? Podemos simplemente pasar una lista:

In [42]:
fq['nueva_col2'] = np.linspace(1, 21, 21)
fq.head()

,Temp,NH4,NO3,OD,Prof,Trans,Caud,SST,STD,PO4,DBO5,DQO,nueva_col,nueva_col2
S1,16.9,0.78,1.83,7.50,27,30,1.01,10.0,86.9,0.6,23.27,29,None,1.0
S2,17.2,0.75,2.44,6.05,56,56,0.68,4.0,87.9,0.4,24.11,5,None,2.0
S3,17.4,1.25,1.60,5.84,37,29,1.53,6.5,101.5,0.6,20.63,15,None,3.0
S4,18.0,0.54,2.95,7.65,122,50,2.46,4.5,91.7,0.7,25.19,7,None,4.0
S5,19.2,0.88,2.70,4.07,137,18,6.58,50.0,95.7,0.2,24.83,16,None,5.0


<div class = "alert alert-block alert-info">
    <p>En la siguiente sesión verás más a profundidad el uso del módulo <code>numpy</code></p></div>

Mencionábamos que también se podía realizar operaciones con los valores de las columnas. Podemos, por ejemplo, *centrar* una columna; es decir, a cada valor de la columna restarle la media (el promedio) de esta. Podríamos calcular el promedio de manera independiente, utilizando la función `np.mean()`, por ejemplo, pero `pandas` incluye métodos para estos casos:

In [43]:
fq['SST_cent'] = fq['SST']-fq['SST'].mean()
fq.head()

,Temp,NH4,NO3,OD,Prof,Trans,Caud,SST,STD,PO4,DBO5,DQO,nueva_col,nueva_col2,SST_cent
S1,16.9,0.78,1.83,7.50,27,30,1.01,10.0,86.9,0.6,23.27,29,None,1.0,-244.299698
S2,17.2,0.75,2.44,6.05,56,56,0.68,4.0,87.9,0.4,24.11,5,None,2.0,-250.299698
S3,17.4,1.25,1.60,5.84,37,29,1.53,6.5,101.5,0.6,20.63,15,None,3.0,-247.799698
S4,18.0,0.54,2.95,7.65,122,50,2.46,4.5,91.7,0.7,25.19,7,None,4.0,-249.799698
S5,19.2,0.88,2.70,4.07,137,18,6.58,50.0,95.7,0.2,24.83,16,None,5.0,-204.299698


Hasta este punto hemos agregado ya tres columnas, pero dos de ellas no son útiles, entonces habrá que removerlas. Para ello podemos utilizar el método `.drop()` que vimos antes, al que le pasaremos una lista con los nombres de las columnas que no nos interesan:

In [44]:
fq.drop(columns = ['nueva_col', 'nueva_col2'], inplace = True)
fq.head()

,Temp,NH4,NO3,OD,Prof,Trans,Caud,SST,STD,PO4,DBO5,DQO,SST_cent
S1,16.9,0.78,1.83,7.50,27,30,1.01,10.0,86.9,0.6,23.27,29,-244.299698
S2,17.2,0.75,2.44,6.05,56,56,0.68,4.0,87.9,0.4,24.11,5,-250.299698
S3,17.4,1.25,1.60,5.84,37,29,1.53,6.5,101.5,0.6,20.63,15,-247.799698
S4,18.0,0.54,2.95,7.65,122,50,2.46,4.5,91.7,0.7,25.19,7,-249.799698
S5,19.2,0.88,2.70,4.07,137,18,6.58,50.0,95.7,0.2,24.83,16,-204.299698


Podemos también renombrar nuestras columnas. Pensemos que el nombre de nuestra columna `SST_cent` lo queremos pasar a `SSTc` para reducir el número de caractéres. Para esto usaremos el método `.rename()`, al cual le pasaremos un diccionario:

In [45]:
fq.rename(columns = {'SST_cent': 'SSTc'}, inplace = True)
fq.head()

,Temp,NH4,NO3,OD,Prof,Trans,Caud,SST,STD,PO4,DBO5,DQO,SSTc
S1,16.9,0.78,1.83,7.50,27,30,1.01,10.0,86.9,0.6,23.27,29,-244.299698
S2,17.2,0.75,2.44,6.05,56,56,0.68,4.0,87.9,0.4,24.11,5,-250.299698
S3,17.4,1.25,1.60,5.84,37,29,1.53,6.5,101.5,0.6,20.63,15,-247.799698
S4,18.0,0.54,2.95,7.65,122,50,2.46,4.5,91.7,0.7,25.19,7,-249.799698
S5,19.2,0.88,2.70,4.07,137,18,6.58,50.0,95.7,0.2,24.83,16,-204.299698


### Indización pt. 2: índices multi-nivel


Un caso sumamente común es el trabajar con datos que están anidados. Usualmente, estos datos están en formato largo o codificado; es decir, tenemos columnas con variables de agrupamiento, en las cuales se repite la etiqueta o el nombre de cada grupo cuantos datos haya en el grupo; es decir, si tenemos 10 individuos con la etiqueta `'Stenella attenuata'`, esa etiqueta aparecerá 10 veces. Esto es sumamente útil al trabajar con los datos (y más adelante veremos algunos ejemplos), pero a veces complica su visualización y la indización. `Pandas` permite utilizar índices multi-nivel (jerárquicos/anidados) que nos permiten indizar rápidamente nuestros datos, como si de una base de datos relacional se tratara. Veamos un ejemplo. Primero, carguemos los datos:

In [46]:
guppy = pd.read_csv('data/Datos1.csv')
guppy.head()

,DIETA,PERIODO,REP,LT,PT
0,A,I,A,0.883,0.50
1,A,I,A,0.909,0.52
2,A,I,A,1.018,0.58
3,A,I,A,0.909,0.52
4,A,I,A,1.200,0.68


Estos datos tenemos mediciones de longitud total y peso total de algunos individuos de guppys, repartidos en tres dietas y tres periodos, con tres réplicas cada uno. Formemos entonces un índice compuesto por esas columnas. Para esto utilizaremos el método `.set_index()`, el cual buscará los valores únicos de cada columna (en órden) y los anidará:

In [47]:
guppy_mi = guppy.copy().set_index(['DIETA', 'PERIODO', 'REP'])
guppy_mi

LT    PT
DIETA PERIODO REP             
A     I       A    0.883  0.50
              A    0.909  0.52
              A    1.018  0.58
              A    0.909  0.52
              A    1.200  0.68
...                  ...   ...
C     F       B      NaN   NaN
              B      NaN   NaN
              B      NaN   NaN
              B      NaN   NaN
              B      NaN   NaN

[180 rows x 2 columns]

¿Cuál es la ventaja? De entrada, ya no tenemos tantos valores repetidos, pero más allá de la estética, nos permite indizar utilizando el atributo `.loc[]`. Extraigamos entonces los individuos de la dieta A, en el periodo final (F) del experimento:

In [48]:
guppy_mi.loc['A', 'F']

<ipython-input-48-456a64186977>:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  guppy_mi.loc['A', 'F']


,LT,PT
REP,,
A,2.1855,1.25
A,2.2817,1.30
A,1.9214,1.10
A,2.3570,1.34
A,2.6166,1.49
A,2.1524,1.23
A,2.3010,1.31
A,2.5875,1.48
A,2.1887,1.25


<div class = "alert alert-block alert-info">
    <p>La advertencia nos dice que el índice no está clasificado/tipificado; es decir, que <code>pandas</code> no sabe exactamente cuáles son los elementos que conforman cada índice, por lo que las operaciones son más lentas. Esto se soluciona con el método <code>.sort_index()</code></p></div>

En nuestro `DataFrame` sin el índice compuesto tendríamos que hacer un proceso bastante más elaborado, el cuál involucra dos máscaras booleanas (una para cada columna y el nivel de interés), encadenadas con el operador `&` (queremos que se cumplan ambas características) y, por último, eliminar las columnas `'DIETA'` y `'PERIODO'`:

In [49]:
guppy[(guppy['DIETA'] == 'A')&(guppy['PERIODO'] == 'F')].drop(columns = ['DIETA', 'PERIODO'])

,REP,LT,PT
120,A,2.1855,1.25
121,A,2.2817,1.30
122,A,1.9214,1.10
123,A,2.3570,1.34
124,A,2.6166,1.49
125,A,2.1524,1.23
126,A,2.3010,1.31
127,A,2.5875,1.48
128,A,2.1887,1.25
129,B,2.3113,1.32


No solo es escribir más, sino que también es más complicado para la computadora.

## Datos faltantes

Habrás notado que en el ejemplo anterior hay algunos valores marcados como `NaN`. Este es el tipo que asigna `NumPy` a los valores faltantes, mientras que `Pandas` lo hace como `None`. Puede ser que las mediciones de ese valor estuvieron incorrectas, o simplemente no se realizaron. Sea cual sea el caso, no es extraño que nuestros datos tengan huecos, y es entonces necesario saber cómo contender con ellos. Una de las operaciones que podemos querer realizar es **imputar** los datos faltantes; es decir, rellenar esos huecos con algún valor. Para ello podemos utilizar el método `.fillna()`:

In [50]:
guppy_mi.loc['A', 'F'].fillna(value = 0)

<ipython-input-50-2dca2ae8cdb7>:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  guppy_mi.loc['A', 'F'].fillna(value = 0)


,LT,PT
REP,,
A,2.1855,1.25
A,2.2817,1.30
A,1.9214,1.10
A,2.3570,1.34
A,2.6166,1.49
A,2.1524,1.23
A,2.3010,1.31
A,2.5875,1.48
A,2.1887,1.25


Podemos también repetir valores que estén en nuestros datos. Con el valor siguiente:

In [51]:
guppy_mi.loc['A', 'F'].fillna(method = 'bfill')

<ipython-input-51-dc78b7289842>:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  guppy_mi.loc['A', 'F'].fillna(method = 'bfill')


,LT,PT
REP,,
A,2.1855,1.25
A,2.2817,1.30
A,1.9214,1.10
A,2.3570,1.34
A,2.6166,1.49
A,2.1524,1.23
A,2.3010,1.31
A,2.5875,1.48
A,2.1887,1.25


O con el valor anterior:

In [52]:
guppy_mi.loc['A', 'F'].fillna(method = 'ffill')

<ipython-input-52-083df118257c>:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  guppy_mi.loc['A', 'F'].fillna(method = 'ffill')


,LT,PT
REP,,
A,2.1855,1.25
A,2.2817,1.30
A,1.9214,1.10
A,2.3570,1.34
A,2.6166,1.49
A,2.1524,1.23
A,2.3010,1.31
A,2.5875,1.48
A,2.1887,1.25


<div class = "alert alert-block alert-danger">
    <p>La imputación de valores faltantes no es un tema trivial, ni tampoco es algo que vayamos a abordar en este curso. Ten mucho cuidado al realizarlo, pues hecho de manera incorrecta es un "cuchareo" o manipulación de los datos.</p></div>

Y, por supuesto, podemos también eliminarlos por completo, para lo que utilizaremos el método `.dropna()`

In [53]:
guppy_mi.loc['A', 'F'].dropna()

<ipython-input-53-d4ab7629e367>:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  guppy_mi.loc['A', 'F'].dropna()


,LT,PT
REP,,
A,2.1855,1.25
A,2.2817,1.30
A,1.9214,1.10
A,2.3570,1.34
A,2.6166,1.49
A,2.1524,1.23
A,2.3010,1.31
A,2.5875,1.48
A,2.1887,1.25


Con esto terminamos la parte introductoria a `pandas`. Revisamos las dos estructuras más básicas (`Series` y `DataFrame`) y vimos cómo a) acceder a sus elementos, b) modificar sus valores y c) realizar operaciones básicas con ellos. En la siguiente sesión veremos algunas operaciones más avanzadas, pero igual de útiles.

<div class = "alert alert-block alert-success">
    <p>FIN DE LA SESIÓN 1 de <code>pandas</code></p></div>

# Información tabular: `Pandas` avanzado

En la sesión anterior revisamos la base del módulo `Pandas` y cómo realizar las operaciones más comunes al tener datos tabulares, pero eso no es todo lo que nos ofrece. En esta sesión veremos cómo hacer nuestro código "*pandorable*", cómo manipular nuestros `DataFrame`, en términos de reducción/procesamiento de datos, y cómo unir distintos `DataFrame`s en uno solo.

## ¿*Pandorable*? ¿idiomático?

Como vimos al final de la sesión anterior, hay muchas maneras de resolver un problema, y siempre hay algunas que son más apropiadas que otras. En `Python`, las mejores soluciones son denominadas **idiomáticas**, las cuales usualmente tienen una alta eficiencia computacional y son fáciles de leer (algunas veces se pierde alguno de los atributos).

`Pandas` también tiene sus soluciones idiomáticas, que se denominan de manera particular **Pandorables**. Un ejemplo es preferir métodos vectorizados a ciclos iterativos, pero el núcleo de un código pandorable es el encadenamiento de métodos. La idea es que cáda método lleva consigo una referencia al objeto al que fue aplicado, por lo que podemos condensar muchas operaciones en un `DataFrame`, incluso en una sola línea de código. De hecho, en la sesión anterior vimos algunos ejemplos de esto, pero hagamos un caso un poco más complejo:

In [54]:
(guppy.where((guppy['LT'] > 1) & (guppy['PT'] > 0.6))
      .dropna()
      .set_index(['DIETA', 'PERIODO'])
      .rename(columns = {'REP': 'REPLICA'})
      .loc['A']
      .head())

,REPLICA,LT,PT
PERIODO,,,
I,A,1.200,0.68
I,B,1.134,0.65
I,B,1.107,0.63
M,A,1.700,0.97
M,A,1.432,0.82


<div class = "alert alert-block alert-info">
    <p>El poner todas las operaciones dentro de un paréntesis nos permite partir el código en más de una línea, haciéndolo más legible</p></div>

Mientras que la forma no pandorable sería:

In [55]:
guppy2 = guppy.copy()
guppy2 = guppy2.where((guppy2['LT'] > 1) & (guppy2['PT'] > 0.6))
guppy2.dropna(inplace = True)
guppy2.set_index(['DIETA', 'PERIODO'], inplace = True)
guppy2.rename(columns = {'REP': 'REPLICA'}, inplace = True)
guppy2.loc['A'].head()

,REPLICA,LT,PT
PERIODO,,,
I,A,1.200,0.68
I,B,1.134,0.65
I,B,1.107,0.63
M,A,1.700,0.97
M,A,1.432,0.82


## Reducir/procesar datos

Siguiendo con el tema de hacer el código pandorable, hay ocasiones en las que queremos aplicar una función a nuestros datos, pero obtener un resultado para cada grupo. Obtener el número de observaciones, su promedio y su desviación estándar es el ejemplo (tal vez) más cotidiano. Aunque definitivamente podemos lograrlo con máscaras booleanas o utilizando índices (simples o multinivel), eso no es pandorable, pero podemos valernos de los métodos `.groupby()`, `apply()` y `agg()`.

### Agrupar datos: `groupby()`

El primer paso lógico siempre va a ser separar nuestros datos según sus grupos, y el método `groupby` sirve para justamente eso. Veámoslo en acción con nuestros datos:

In [56]:
guppy.groupby(['DIETA', 'PERIODO'])

¿Qué sucedió? Es bastante posible que esperaras una salida similar a utilizar `.set_index(['DIETA', 'PERIODO'])`, o tal vez dos tablas separadas, pero definitivamente no lo que sea que salió arriba. Pues bien, esto tiene que ver con lo que mencionamos antes: los métodos aplicados a un objeto regresan una referencia. No es que no se hayan agrupado los datos, sino que se regresó un marcador a los datos agrupados. Podemos entonces encadenar la operación que nosotros querramos aplicar a cada grupo, obtener su promedio, por ejemplo:

In [57]:
guppy.groupby(['DIETA', 'PERIODO']).mean()

LT        PT
DIETA PERIODO                    
A     F        2.154142  1.228947
      I        0.956400  0.546000
      M        1.478526  0.844211
B     F        2.042417  1.165000
      I        0.977050  0.557000
      M        1.471722  0.840000
C     F        2.019167  1.151111
      I        0.963600  0.550000
      M        1.497733  0.855333

Aquí la salida fue un `DataFrame` con un índice multinivel, en el que nos dió el promedio de ambas variables para cada periodo dentro de cada dieta. ¿Y si quiero saber el número de observaciones? Para eso está el método `.count()`:

In [58]:
guppy.groupby(['DIETA', 'PERIODO']).count()

REP  LT  PT
DIETA PERIODO             
A     F         20  19  19
      I         20  20  20
      M         20  19  19
B     F         20  18  18
      I         20  20  20
      M         20  18  18
C     F         20   9   9
      I         20  20  20
      M         20  15  15

¿Y la desviación estándar? Con el método `.std()`:

In [59]:
guppy.groupby(['DIETA', 'PERIODO']).std()

LT        PT
DIETA PERIODO                    
A     F        0.281098  0.160966
      I        0.108496  0.061422
      M        0.149505  0.085526
B     F        0.243320  0.138702
      I        0.138680  0.080138
      M        0.095295  0.054124
C     F        0.187717  0.105646
      I        0.111901  0.064236
      M        0.090782  0.051667

Ahora te estarás preguntando, ¿puedo unir estas tres tablas en una sola? Pues sí, sí que puedes, pero primero veamos cómo aplicar cualquier función a nuestros datos (agrupados o no).

### Funciones a todas las columnas o todos los renglones: `.apply()`

Imagina el siguiente escenario: tienes una función personalizada que quieres aplicar a todos los renglones de alguna(s) columna(s) de tu `DataFrame`. Una transformación (ejem, deformación, ejem) logarítmica, por ejemplo. Puedes aplicar la función directamente a cada columna, tal que:

In [60]:
np.log(guppy['LT'])

0     -0.124430
1     -0.095410
2      0.017840
3     -0.095410
4      0.182322
         ...   
175         NaN
176         NaN
177         NaN
178         NaN
179         NaN
Name: LT, Length: 180, dtype: float64

Pero esto evidentemente no es eficiente ni sencillo en el momento en el que esto escala a más columnas. ¿Qué podemos hacer? el método `.apply()` nos permite aplicar una función a todos los elementos de un renglón o una columna de un `DataFrame`, de manera vectorizada. Una forma sencilla de demostrarlo es obteniendo el valor máximo de cada renglón de nuestros parámetros fisicoquímicos (ojo, no tiene sentido, es solo para fines demostrativos):

In [61]:
fq.apply(lambda x: np.max(x), axis = 1)

S1       86.900000
S2       87.900000
S3      101.500000
S4      122.000000
S5      137.000000
S6      168.000000
S7      148.000000
S8     1163.333333
S9      298.666667
S10     572.000000
S11     649.000000
S12     599.000000
S13     199.000000
S14     607.594937
S15     134.000000
S16     129.100000
S17     137.700000
S18     234.615385
S19     160.000000
S20     153.333333
S21     290.000000
dtype: float64

Ahora expliquemos qué fue lo que hicimos:

1. Aplicamos el método `apply` a nuestro `DataFrame` `fq`
2. La función a aplicar es una función anónima (`lambda`) que recibe un solo argumento: `x`, para el cual obtendrá el número máximo.
3. `x` es cada renglón del `DataFrame`, lo cual indicamos con el argumento `axis = 1`. ¿Por qué 1, si queremos extraer el máximo de cada rengón y el eje 1 indica las columnas? Porque es el eje **sobre** el cual queremos que aplique la función. Al haber dicho que queremos que lo aplique sobre las columnas, lo que `.apply()` está haciendo es extraer todos los valores de todas las columnas, renglón a renglón. Cada uno de estos renglones sustituye a x y, por tanto, se le aplica la función `np.max()`, que obtiene el número máximo.

Este último paso es un poco complejo, lo sé, pero se resume a que si queremos aplicar la función a cada renglón (con todas sus columnas) utilizaremos `axis = 1`, caso contrario (cada columna con todos sus renglones) utilizaremos `axis = 0`.

### Múltiples funciones a datos agrupados: `.agg()`

Ahora sí, volvamos a nuestro problema de obtener una tabla descriptiva de datos agrupados; es decir, obtener el número de observaciones, la media y la desviación estándar de cada grupo. Si bien podríamos resolverlo combinando `.groupby().apply()` y declarando una función que reciba los datos de una columna y que regrese sus descriptores, podemos simplificarnos la existencia y utilizar el método `.agg()`. Este método recibe un diccionario, en donde las etiquetas son los nombres de las columnas a las que queremos aplicar las funciones y los valores son las funciones que queremos aplicar. Si las funciones tienen métodos incluidos en `Pandas`, podemos pasar únicamente sus nombres, caso contrario, habrá que pasar la función correspondiente SIN PARÉNTESIS NI ARGUMENTOS (los argumentos los podemos pasar directamente en `.agg`):

In [62]:
funs = ['count', 'mean', 'std', np.max, np.min]
guppy.dropna().agg({'LT': funs,
                    'PT': funs})

,LT,PT
count,158.000000,158.000000
mean,1.461106,0.833734
std,0.485325,0.276816
amax,2.616600,1.490000
amin,0.600000,0.340000


Y esto mismo lo podemos pasar a los datos agrupados:

In [63]:
desc = guppy.groupby(['DIETA','PERIODO']).agg({'LT': funs,
                                               'PT': funs})
desc

LT                                        PT            \
              count      mean       std    amax    amin count      mean   
DIETA PERIODO                                                             
A     F          19  2.154142  0.281098  2.6166  1.3735    19  1.228947   
      I          20  0.956400  0.108496  1.2000  0.8000    20  0.546000   
      M          19  1.478526  0.149505  1.8000  1.1000    19  0.844211   
B     F          18  2.042417  0.243320  2.5119  1.5840    18  1.165000   
      I          20  0.977050  0.138680  1.2000  0.6000    20  0.557000   
      M          18  1.471722  0.095295  1.6170  1.2810    18  0.840000   
C     F           9  2.019167  0.187717  2.3039  1.7186     9  1.151111   
      I          20  0.963600  0.111901  1.2220  0.7730    20  0.550000   
      M          15  1.497733  0.090782  1.6430  1.3080    15  0.855333   

                                     
                    std  amax  amin  
DIETA PERIODO                        
A     F        0.160966  1.49  0.78  
      I        0.061422  0.68  0.46  
      M        0.085526  1.03  0.63  
B     F        0.138702  1.43  0.90  
      I        0.080138  0.68  0.34  
      M        0.054124  0.92  0.73  
C     F        0.105646  1.31  0.98  
      I        0.064236  0.70  0.44  
      M        0.051667  0.94  0.75

Notarás que aquí la tabla ya es más compleja, pues ahora tenemos un índice multinivel para nuestros grupos, pero también en nuestras variables. Esto quiere decir que podemos extraer los valores que nos interesen de manera rápida. Por ejemplo, los descriptivos de la dieta A en el periodo F:

In [64]:
desc.loc['A', 'F']

LT  count    19.000000
    mean      2.154142
    std       0.281098
    amax      2.616600
    amin      1.373500
PT  count    19.000000
    mean      1.228947
    std       0.160966
    amax      1.490000
    amin      0.780000
Name: (A, F), dtype: float64

O el promedio en `'PT'` de la dieta A y el periodo F:

In [65]:
desc.loc['A', 'F'].loc['PT', 'mean']

1.2289473684210528

## Unir `DataFrames`

El proceso de unir `DataFrames` es más complicado de lo que realizamos al añadir nuevas columnas o nuevos valores. Para ello utilizaremos el método `.merge(DF1, DF2, how, left_index, right_index)`. Este método está basado en la teoría relacional, y por lo tanto, podemos utilizar un diagrama de Venn para ejemplificarlo: